In [1]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

## Helpers

In [2]:
secrets_resolver = SecretsResolverDemo()

In [3]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [4]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Receiving the OOB message

In [106]:
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))



## Preparing message

In [107]:
bob_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/bob")

In [97]:
# bob_response_message = Message(
#     body = { "response_requested": True },
#     custom_headers = [{"return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/trust-ping/2.0/ping",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]]
# )
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all",
#         "created_time": int(datetime.datetime.now().timestamp())            
#                       }],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/basicmessage/2.0/message",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {"content": "Who won the soccer world cup in 1986?"}
# )
# bob_response_message = Message(
#     custom_headers = [{"return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/coordinate-mediation/2.0/keylist-query",
#     body = {}
# )
# bob_response_message = Message(
#     custom_headers = [{"return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
#     body = {
#         "updates": [
#             {
#             "recipient_key": "did:key:z6MkpTHR8VNsBxYAAWHut2Geadd9jSwuBV8xRoAnwWsdvktH2",
#             "action": "remove"
#             }
#         ]
            
#     }
# )
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/discover-features/2.0/queries",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {
#         "queries": [
#             { "feature-type": "protocol", "match": "https://didcomm.org/basicmessage/2.*" }
#         ]
#     }
# )
bob_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/shorten-url/1.0/request-shortened-url",
    frm = bob_did,
    to = [received_msg_decoded["from"]],
    body = {
            "url": "https://my.example.com/superlongurl",
            "requested_validity_seconds": 86400,
            "goal_code": "shorten.oobv2",
#             "short_url_slug": "algo"
    }
)
# bob_response_message = Message(
#     custom_headers = [{
#         "return_route": "all"}],
#     id = str(uuid.uuid4()),
#     #pthid = received_msg_decoded["id"],
#     type = "https://didcomm.org/shorten-url/1.0/invalidate-shortened-url",
#     frm = bob_did,
#     to = [received_msg_decoded["from"]],
#     body = {
#             "shortened_url": "http://127.0.0.1:8000/qr?_oobid=eaf4166322d04ddba2829f0807b6a7cf",
#     }
# )


In [98]:
bob_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = bob_response_message,
    frm = bob_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

In [109]:
alice_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
alice_endpoint = alice_did_doc["service"][0]["serviceEndpoint"]
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(alice_endpoint, headers=headers, data = bob_packed_msg.packed_msg)


## Decoding response


In [110]:
bob_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

In [111]:
print(bob_unpack_msg)

UnpackResult(message=Message(id='d36d926b-cd8e-4fb2-ac58-ebda97c5c14b', type='https://didcomm.org/shorten-url/1.0/shortened-url', body={'shortened_url': 'https://mediator.rootsid.cloud/qr?_oobid=c343cd62df8a4b4e938553486843991b', 'expires_time': 1663059798}, frm=None, to=None, created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid='c6b5b94c-7570-452f-bb4c-db077c3ab6fd', pthid=None, attachments=None, custom_headers=None), metadata=Metadata(encrypted=True, authenticated=True, non_repudiation=False, anonymous_sender=False, re_wrapped_in_forward=False, encrypted_from='did:peer:2.Ez6LSgtdJFaYmabrU5bPFeyT5s9YQs2ZCpDVxF92gc6oNnRQH.Vz6MkirjkZ9X8moByhdSwfhqJ7ezEzCUTdhsSsUScubgYZtNA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL21lZGlhdG9yLnJvb3RzaWQuY2xvdWQiLCJhIjpbImRpZGNvbW0vdjIiXX0#6LSgtdJFaYmabrU5bPFeyT5s9YQs2ZCpDVxF92gc6oNnRQH', encrypted_to=['did:peer:2.Ez6LSrCiekfBn15YEmnkRTFL9x4BHu3PtUEoFaH6WTThwG19L.Vz6MkmU6h1nfHwWeyHSRtLjPkwPKfAGtdo8rBdTJcG7Q4a8oy.S

In [112]:
print(bob_unpack_msg.message)

Message(id='d36d926b-cd8e-4fb2-ac58-ebda97c5c14b', type='https://didcomm.org/shorten-url/1.0/shortened-url', body={'shortened_url': 'https://mediator.rootsid.cloud/qr?_oobid=c343cd62df8a4b4e938553486843991b', 'expires_time': 1663059798}, frm=None, to=None, created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid='c6b5b94c-7570-452f-bb4c-db077c3ab6fd', pthid=None, attachments=None, custom_headers=None)
